In [ ]:
import requests
import pandas as pd
from datetime import datetime, timedelta

# Remplacez 'YOUR_CLIENT_ID' et 'YOUR_CLIENT_SECRET' par vos véritables informations d'authentification
CLIENT_ID = "rFhIuiYJEoRGwtXB0m6LekcFhCta6utw"
CLIENT_SECRET = "t8j81tZlGcFIWFap"

# Obtenez un jeton d'authentification
token_url = "https://test.api.amadeus.com/v1/security/oauth2/token"
token_payload = {
    "grant_type": "client_credentials",
    "client_id": CLIENT_ID,
    "client_secret": CLIENT_SECRET,
}

token_response = requests.post(token_url, data=token_payload)

if token_response.status_code == 200:
    access_token = token_response.json().get("access_token")
    headers = {"Authorization": f"Bearer {access_token}"}
    # Plages de dates
    date_ranges = [
        ("2019-07-01", "2019-09-30"),
        ("2020-07-01", "2020-09-30"),
        ("2021-07-01", "2021-09-30"),
        ("2022-07-01", "2022-09-30"),
        ("2023-07-01", "2023-08-16"),
    ]
    # Boucle sur les plages de dates et enregistrement des données dans des fichiers CSV
    for start_date, end_date in date_ranges:
        current_date = datetime.strptime(start_date, "%Y-%m-%d")
        end_date = datetime.strptime(end_date, "%Y-%m-%d")
        while current_date <= end_date:
            formatted_date = current_date.strftime("%Y-%m-%d")
            API_URL = f"https://test.api.amadeus.com/v1/analytics/itinerary-price-metrics?originIataCode=MAD&destinationIataCode=CDG&departureDate={formatted_date}&currencyCode=EUR&oneWay=false"
            response = requests.get(API_URL, headers=headers)
            
            if response.status_code == 200:
                api_data = response.json()
                if (
                    "data" in api_data
                    and isinstance(api_data["data"], list)
                    and len(api_data["data"]) > 0
                ):
                    item = api_data["data"][0]  # Get the first item from the list
                    # Extract data from the item dictionary
                    price_metrics = item.get("priceMetrics", [])
                    records = []
                    for metric in price_metrics:
                        record = {
                            "date": formatted_date,
                            "amount": metric.get("amount"),
                            "quartileRanking": metric.get("quartileRanking"),
                        }
                        records.append(record)
                    df = pd.DataFrame(records)
                    year = current_date.year
                    csv_file = f"{year}.csv"
                    df.to_csv(csv_file, index=False, mode="a", header=not current_date.month > 1)
                    print(f"Data saved to {csv_file}")

                current_date += timedelta(days=1)
            else:
                print(f"Failed to fetch data for date {formatted_date}. Status code: {response.status_code}")
else:
    print(f"Failed to get access token. Status code: {token_response.status_code}")
